In [ ]:
import polars as pl
import validate.lesion as vl
import validate.general as vg

from constants import list_id_cols

import utils
from pathlib import Path
import os

PATH_INPUT = os.getenv("PATH_INPUT")

lf = utils.get_df(Path(PATH_INPUT).joinpath("test data.accdb")).lazy()

lf2 = utils.get_df(Path(PATH_INPUT).joinpath("ZON CHERAS OC DATABASE JULY-OCT 2023 edit MAIwani.accdb")).lazy()

obj = vg.ValidationGeneral(lf, "test data")
lfall = pl.concat([lf, lf2])

In [ ]:
lfall.columns

In [ ]:
lfall.select("FAMILY", "FAMILYHSIT SPECIFY", "RELATION").collect()

In [ ]:
famihistcancer_whitelist = [
  "false|false|false",
  "true|true|true"
]

lfall.select("FAMILY", "FAMILYHSIT SPECIFY", "RELATION").with_columns(
  pl.when(pl.col("FAMILYHSIT SPECIFY").is_not_null())
  .then(True)
  .otherwise(False)
  .alias("specify_filled"),
  pl.when(pl.col("RELATION").is_not_null())
  .then(True)
  .otherwise(False)
  .alias("relation_filled"),
).with_columns(
  pl.concat_str("FAMILY", "specify_filled", "relation_filled", separator="|").alias(
    "invalid_combination"
  )
).filter(~pl.col("invalid_combination").is_in(famihistcancer_whitelist)).collect()
